# Notebook magics
We start a cluster as done in the [Introduction](./intro.ipynb)

In [1]:
import ipyparallel as ipp
cluster = ipp.Cluster(n=4, engines="mpi")
rc = cluster.start_and_connect_sync(activate=True)

Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/4 [00:00<?, ?engine/s]

We can now use the %%px magic to use this cluster in a notebook cell

In [2]:
%%px
from mpi4py import MPI
import os
import time
start = time.time()
rank = MPI.COMM_WORLD.rank
pid = os.getpid()
if rank == 0:
    time.sleep(2)
print(f"{time.time()-start:.2e} Hello I am process {rank} with Unix ID {pid}")

[stdout:1] 2.45e-04 Hello I am process 1 with Unix ID 38827


[stdout:2] 2.36e-04 Hello I am process 2 with Unix ID 38828


[stdout:3] 4.01e-04 Hello I am process 3 with Unix ID 38829


%px:   0%|          | 0/4 [00:00<?, ?tasks/s]

[stdout:0] 2.00e+00 Hello I am process 0 with Unix ID 38826


You have all variables defined in a %%px cell in the next cell declared with this magic as well 

In [3]:
%%px
print(rank)

[stdout:0] 0


[stdout:1] 1


[stdout:2] 2


[stdout:3] 3


Note that we have not imported `mpi4py` outside of the cluster

In [4]:
try:
    print(MPI.COMM_WORLD.size)
except NameError as e:
    print(f"Error: {e}")

Error: name 'MPI' is not defined


If you only want a single line to be executed on the cluster, you can use the `%px` magic.

In [5]:
import os as osc
from mpi4py import MPI as MPIc
print(f"Controller process pid: {osc.getpid()} {MPIc.COMM_WORLD.rank}/{MPIc.COMM_WORLD.size}")
%px print(f"Engine pid: {os.getpid()} {MPI.COMM_WORLD.rank}/{MPI.COMM_WORLD.size}")


Controller process pid: 38747 0/1


[stdout:1] Engine pid: 38827 1/4


[stdout:0] Engine pid: 38826 0/4


[stdout:3] Engine pid: 38829 3/4


[stdout:2] Engine pid: 38828 2/4


A nicer way to do this would be with

In [6]:
with rc[:].sync_imports():
    from mpi4py import MPI
    import os

print(f"Controller process pid: {os.getpid()} {MPI.COMM_WORLD.rank}/{MPI.COMM_WORLD.size}")
%px print(f"Engine pid: {os.getpid()} {MPI.COMM_WORLD.rank}/{MPI.COMM_WORLD.size}")


importing MPI from mpi4py on engine(s)
importing os on engine(s)
Controller process pid: 38747 0/1


[stdout:1] Engine pid: 38747 0/1


[stdout:0] Engine pid: 38747 0/1


[stdout:3] Engine pid: 38747 0/1


[stdout:2] Engine pid: 38747 0/1
